In [1]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')

values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

playing = True

In [2]:
class Card:
    
    def __init__(self, suit, rank):
        
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return self.rank +'  of  ' + self.suit

In [3]:
class Deck:
            
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))  # build Card objects and add them to the list
                
    def __str__(self):
        deck_comp = ''  # start with an empty string
        for card in self.deck:
            deck_comp += '\n ' +card.__str__() # add each Card object's print string
        return 'The Deck has: '+ deck_comp
    
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        single_card = self.deck.pop()
        return single_card                

In [4]:
test_deck = Deck()

In [5]:
class Hand:
    
    def __init__(self):
        self.cards = []   # start with an empty list as we did in the Deck class
        self.value = 0    # start with zero value
        self.aces = 0      # add an attribute to keep track of aces
        
    def add_card(self,card):
        self.cards.append(card)
        self.value += values[card.rank]
        if card.rank == 'Ace':
            self.aces += 1  # add to self.aces
        
    def adjust_for_ace(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1

In [6]:
test_deck = Deck()
test_deck.shuffle()
test_player = Hand()
test_player.add_card(test_deck.deal())
test_player.add_card(test_deck.deal())
test_player.value

15

In [7]:
for card in test_player.cards:
    print(card)

Five  of  Spades
Ten  of  Hearts


In [8]:
class Chips:
    
    def __init__(self):
        self.total = int(input('What should be the maximum betting limit?  '))
        self.bet = 0
        
    def win_bet(self):
        self.total += self.bet
        
    def lose_bet(self):
        self.total -= self.bet

In [9]:
def take_bet(chips):
    
    while True:
        try:
            chips.bet = int(input('Value of your chips to bet:  '))
        except ValueError:
            print('Sorry! The bet number must be in digits')
        else:
            if chips.bet > chips.total:
                print('Sorry!, You can not exceed',chips.total)
            else:
                break

In [10]:
def hit(deck,hand):
    
    hand.add_card(deck.deal())
    hand.adjust_for_ace()

In [11]:
def hit_or_stand(deck,hand):
    global playing  # to control an upcoming while loop
    
    while True:
        x = input("Would you like to hit or stand? Enter 'h' or 's' ")
        
        if x[0].lower() == 'h':
            hit(deck,hand)   # hit() function defined above
        elif x[0] == 's':
            print('Player stands. The dealer is playing')
            playing = False
        else:
            print('Sorry! Please try again.')
            continue
        break

In [12]:
def show_some(player,dealer):
    print("\nDealer's Hand:")
    print(" <card hidden> ")
    print('',dealer.cards[1])
    print("\nPlayer's Hand:",*player.cards, sep='\n ')
    
def show_all(player,dealer):
    print("\nDealer's Hand:", *dealer.cards, sep='\n ')
    print("Dealer's Hand =",dealer.value)
    print("\nPlayer's Hand:", *player.cards, sep='\n ')
    print("Player's Hand =",player.value)

In [13]:
def player_busts(player,dealer,chips):
    print('Player has lost')
    chips.lose_bet()
    
def player_wins(player,dealer,chips):
    print("player Wins")
    chips.win_bet()
    
def dealer_busts(player,dealer,chips):
    print('Dealer loses')
    chips.win_bet()
    
def dealer_wins(player,dealer,chips):
    print('Dealer Wins')
    chips.lose_bet()
    
def push(player,dealer):
    print("Dealer and Player tie. It's a push")

In [14]:
while True:
     # Print an opening statement
    print('\nWelcome to BlackJack! Get as close to 21 as you can without going over!\n\
    Dealer hits until she reaches 17. Aces count as 1 or 11.\n')
    
    # Create & shuffle the deck, deal two cards to each player
    
    deck = Deck()
    deck.shuffle()
    
    player_hand = Hand()
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand = Hand()
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    # Set up the Player's chips
    
    player_chips = Chips()
    
    # Prompt the Player for their bet
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player_hand,dealer_hand)
    
    while playing:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        hit_or_stand(deck,player_hand)
        
        # Show cards (but keep one dealer card hidden)
        show_some(player_hand,dealer_hand)
        
         # If player's hand exceeds 21, run player_busts() and break out of loop
        if player_hand.value > 21:
            player_busts(player_hand,dealer_hand,player_chips)
            break
        
        # If Player hasn't busted, play Dealer's hand until Dealer reaches 17 
        
        if player_hand.value < 21:
            
            while dealer_hand.value < 17:
                hit(deck,dealer_hand)
                
            # Show all cards
            show_all(player_hand,dealer_hand)
            
            # Run different winning scenarios
            if dealer_hand.value > 21:
                dealer_busts(player_hand,dealer_hand,player_chips)
                
            elif dealer_hand.value > player_hand.value:
                dealer_wins(player_hand,dealer_hand,player_chips)
                
            elif dealer_hand.value < player_hand.value:
                player_wins(player_hand,dealer_hand,player_chips)
                
            else:
                push(player_hand,dealer_hand)
                
            # Inform Player of their chips total
        print("\nPlayer's winning stand at: ",player_chips.total)
        
        # Ask to play again
    new_game = input("Would you like to play another hand? Enter 'y' or 'n' ")
        
    if new_game[0].lower() == 'y':
        playing = True
        continue
            
    else:
        print('\nThanks for playing at the Belagio Casino')
    break


Welcome to BlackJack! Get as close to 21 as you can without going over!
    Dealer hits until she reaches 17. Aces count as 1 or 11.

What should be the maximum betting limit?  10000
Value of your chips to bet:  10000

Dealer's Hand:
 <card hidden> 
 Four  of  Spades

Player's Hand:
 Ten  of  Hearts
 Jack  of  Clubs
Would you like to hit or stand? Enter 'h' or 's' s
Player stands. The dealer is playing

Dealer's Hand:
 <card hidden> 
 Four  of  Spades

Player's Hand:
 Ten  of  Hearts
 Jack  of  Clubs

Dealer's Hand:
 Queen  of  Clubs
 Four  of  Spades
 Ten  of  Diamonds
Dealer's Hand = 24

Player's Hand:
 Ten  of  Hearts
 Jack  of  Clubs
Player's Hand = 20
Dealer loses

Player's winning stand at:  20000
Would you like to play another hand? Enter 'y' or 'n' n

Thanks for playing at the Belagio Casino
